# CGPO Cloud Training Notebook

This notebook allows you to train the Cognitive Graph Portfolio Optimizer on Google Colab's free GPU.

## Instructions
1. Upload your `backend/` folder to Colab (or clone from GitHub)
2. Run all cells in order
3. Download the trained model (`models/agent.pth`) when complete

In [ ]:
# Step 1: Check GPU availability
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

In [ ]:
# Step 2: Install dependencies
!pip install -q yfinance pandas numpy torch torch-geometric textblob ta gymnasium

In [ ]:
# Step 3: Upload your backend folder
# Option A: Upload manually using the file browser on the left
# Option B: Clone from GitHub (uncomment below)

# !git clone https://github.com/YOUR_USERNAME/CGPO-Project.git
# %cd CGPO-Project/backend

In [ ]:
# Step 4: Set up path (adjust if you uploaded manually)
import sys
sys.path.append('/content/backend')  # Adjust path as needed

# If you uploaded manually to /content/backend/
%cd /content/backend

In [ ]:
# Step 5: Import modules
from core.data_loader import MarketDataLoader
from core.market_env import MarketGraphEnv
from core.agent import Agent

print("Modules imported successfully!")

In [ ]:
# Step 6: Configure training
TICKERS = ["AAPL", "MSFT", "GOOGL", "AMZN", "NVDA", "META", "TSLA", "AMD", "INTC", "NFLX"]
TRAINING_EPISODES = 50  # Increase for better results
LEARNING_RATE = 0.001
ENTROPY_COEF = 0.01

In [ ]:
# Step 7: Fetch data and initialize environment
print("Fetching market data...")
loader = MarketDataLoader(TICKERS)
data = loader.fetch_history(period="1y", interval="1d")
print(f"Data shape: {data.shape}")

print("Initializing environment...")
env = MarketGraphEnv(TICKERS, data, window_size=20)
print("Environment ready!")

In [ ]:
# Step 8: Initialize agent
obs, _ = env.reset()
num_features = obs['x'].shape[1]
num_assets = len(TICKERS)

agent = Agent(
    num_features=num_features,
    num_assets=num_assets,
    lr=LEARNING_RATE,
    entropy_coef=ENTROPY_COEF
)
print(f"Agent initialized on: {agent.device}")

In [ ]:
# Step 9: Train!
print(f"Starting training for {TRAINING_EPISODES} episodes...")
rewards = agent.train(env, episodes=TRAINING_EPISODES)
print(f"\nTraining complete! Final avg reward: {sum(rewards[-10:])/10:.4f}")

In [ ]:
# Step 10: Save model
agent.save_model("/content/agent.pth")
print("Model saved to /content/agent.pth")

# Download the model
from google.colab import files
files.download('/content/agent.pth')

In [ ]:
# Step 11: Plot training rewards
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(rewards)
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.title('Training Progress')
plt.grid(True)
plt.show()